# 下面是可能会用到的库

In [4]:
import numpy as np
import pandas as pd
import time
import sys
import datetime
import gc
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from time import time
import Levenshtein
from gensim.models import Word2Vec
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn import datasets

data_path='/home/kesci/input/bytedance/first-round/'
data_store_path='/home/kesci/work/1000W_stacking/'

# 处理数据

## 读测试集

In [3]:
all_data_test_list=[]
for epoch,part_data in enumerate(pd.read_csv(data_path+'test.csv',sep=',',names=['query_id','query','query_title_id','title'],usecols=['query','title'],chunksize=1000000,header=None)):
    all_data_test_list.append(part_data)
test_data=pd.concat(all_data_test_list,axis=0)
test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))#####测试集的title 有\t，需要处理掉
#,usecols=[1,3]
test_data

,query,title
0,11202 184 50256,11202 184 2346 2527 274 383 34 1033 156 18 502...
1,11202 184 50256,11202 184 21479 808 383 34 1033 18 50256 19 27...
2,11202 184 50256,11202 184 21479 15 227 383 34 1033 156 116 18 ...
3,11202 184 50256,11202 184 274 383 34 1033 156 15 18 50256 19 2...
4,1013 6811 14038 1156,361 628 1020 513 126 15 1013 6811 2315 743 27 ...
5,1013 6811 14038 1156,21 126 15 1013 6811 743
6,1013 6811 14038 1156,4303 15 1013 6811 743 18608 202 3474 671 4 280...
7,28 240 2157,28 240 10 11
8,28 240 2157,28 240 10 27 32 241 289 241
9,28 240 2157,28 240 10 27 5922 12478 11


## 读训练集（全量）（最后会成numpy形式）

In [1]:
all_data_train_list=[]
for epoch,part_data in enumerate(pd.read_csv(data_path+"train.csv",names=['query_id','query','query_title_id','title','label'],usecols=[1,3,4],chunksize=1000000,header=None)):
    all_data_train_list.append(part_data)
    print(epoch)
all_data_train=pd.concat(all_data_train_list,axis=0)
del all_data_train_list

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


# 读已经存储好的train_data

In [5]:
# train_data_ctr=pd.read_csv(data_store_path+"train_data_ctr.csv")#读已经做好ctr特征的训练集
# test_data_ctr=pd.read_csv(data_store_path+"test_data_ctr.csv")#读已经做好ctr特征的测试集
# train_data=pd.read_csv(data_store_path+"train_data.csv")#读训练集（已经用shuffle分好了的）
# train_data=pd.concat([train_data,train_data_ctr],axis=1)
# test_data=pd.concat([test_data,test_data_ctr],axis=1)
# del train_data_ctr
# del test_data_ctr
# train_data
##########################################################################################
# train_data=pd.read_csv(data_store_path+"train_feature_first_no_query_and_title_data.csv")#读已经做好ctr特征的训练集
# test_data=pd.read_csv(data_store_path+"test_feature_first_no_query_and_title_data.csv")#读已经做好ctr特征的测试集
###################################################下面这个是为了去做 相似度特征
# train_data=pd.read_csv(data_store_path+"train_feature_first_data.csv")#读已经做好ctr特征的训练集
# test_data=pd.read_csv(data_store_path+"test_feature_first_data.csv")#读已经做好ctr特征的测试集

############################################
train_data=pd.read_csv(data_store_path+"data_sets/train_data_ctr.csv")#读已经做好ctr特征的训练集
test_data=pd.read_csv(data_store_path+"data_sets/test_data_ctr.csv")#读已经做好ctr特征的测试集

# import h5py
# f = h5py.File(data_store_path+"train_data.hd5",'r')   #打开h5文件
# print(f.keys)
# a = f['train']                    #取出主键为data的所有的键值
# f.close()
# print(a)

## 随机取1000W训练集 9000W数据用于做CTR特征

### 这里是因为第一次打比赛不太懂,所以用了StratifiedShuffleSplit来取数据
### 实际上用DataFrame.sample()来取更加简单。
### 但是StratifiedShuffleSplit还是有好处的，他可以保证取到的数据 label  0、1的比例和和原来是相同的

In [5]:
data_split=StratifiedShuffleSplit(n_splits=5,test_size=0.90,random_state=1)
train_index,feature_index=data_split.split(all_data_train['label'],all_data_train['label']).__next__()
print('StratifiedShuffleSplit ended')
# all_data_train=all_data_train.values
train_data=all_data_train.iloc[train_index]
# train_data=pd.DataFrame(train_data,columns=['query','title','label'])#用于训练的数据
feature_data=all_data_train.iloc[feature_index]
# feature_data=pd.DataFrame(feature_data,columns=['query','title','label'])#用于做特征的数据
del all_data_train
# print('start store')
print("开始存训练集")
time1=time()
train_data.to_csv(data_store_path+"data_sets/train_data.csv",index=None)
time2=time()
print("存训练集总共用时：",time2-time1)
del feature_index
del train_index

StratifiedShuffleSplit ended
开始存训练集
存训练集总共用时： 71.92939972877502


In [5]:
# print("开始存CTR特征集集")
# time1=time()
# feature_data[0:20000000].to_csv(data_store_path+"feature_data1.csv",index=None)
# feature_data[20000000:40000000].to_csv(data_store_path+"feature_data2.csv",index=None)
# feature_data[40000000:60000000].to_csv(data_store_path+"feature_data3.csv",index=None)
# feature_data[60000000:80000000].to_csv(data_store_path+"feature_data4.csv",index=None)
# feature_data[80000000:].to_csv(data_store_path+"feature_data5.csv",index=None)
# time2=time()
# print("存CTR特征集总共用时：",time2-time1)

开始存CTR特征集集
存CTR特征集总共用时： 561.6834108829498


### test_data为测试集

# 用9000W数据groupby出来ctr，然后按title拼接到1000W上，做1000W的CTR特征

### 下面的代码用了矩估计方法，做了贝叶斯平滑

In [9]:
# feature_data
def getBayesSmoothParam(origion_rate):
    origion_rate_mean = origion_rate.mean()
    origion_rate_var = origion_rate.var()
    alpha = origion_rate_mean / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    beta = (1 - origion_rate_mean) / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    return alpha, beta
    
feature_data['label']=feature_data['label'].astype(int)
# query_title_ctr=feature_data.groupby(['query','title'], as_index = False)['label'].agg({ 'query_title_count':'count','query_title_ctr':'mean'})
# train_data = pd.merge(train_data, query_title_ctr, on=['query','title'], how='left')
# test_data= pd.merge(test_data, query_title_ctr, on=['query','title'], how='left')
# del query_title_ctr
# ########################################################
# query_ctr=feature_data.groupby('query', as_index = False)['label'].agg({ 'query_count':'count','query_ctr':'mean'})
# feature_data=feature_data.drop(['query'],axis=1)
# train_data = pd.merge(train_data, query_ctr, on='query', how='left')
# test_data= pd.merge(test_data, query_ctr, on='query', how='left')
# del query_ctr
#######################################
title_ctr=feature_data.groupby('title', as_index = False)['label'].agg({ 'title_count':'count','title_ctr':'mean','title_sum':'sum'})
del feature_data
print("ctr基本做完，开始平滑")
alpha, beta = getBayesSmoothParam(title_ctr['title_ctr'])
title_ctr['title_ctr'] = (title_ctr['title_sum'] + alpha) / (title_ctr['title_count'] + alpha + beta)
print("平滑完成")
train_data = pd.merge(train_data, title_ctr, on='title', how='left')
test_data= pd.merge(test_data, title_ctr, on='title', how='left')
del title_ctr

print("保存做好ctr特征的数据")
train_data.to_csv(data_store_path+"data_sets/train_data_ctr.csv",index=None)
test_data.to_csv(data_store_path+"data_sets/test_data_ctr.csv",index=None)
print("保存完毕")

ctr基本做完，开始平滑
平滑完成
保存做好ctr特征的数据
保存完毕


In [6]:
train_data[['query']]=train_data[['query']].applymap(lambda x:x.split(' '))
train_data[['title']]=train_data[['title']].applymap(lambda x:x.split(' '))
test_data[['query']]=test_data[['query']].applymap(lambda x:x.split(' '))
test_data[['title']]=test_data[['title']].applymap(lambda x:x.split(' '))

### 现在train_data为训练数据（包含训练集和测试集，还没切分），feature_data(用于给train_data做CTR特征的数据)

# 新加入的特征

In [ ]:
#词的个数（文本长度）
for feature in ['query', 'title']:
    train_data[feature + '_word_num'] = train_data[feature].apply(lambda x: len(x.split()))


train_data['prefix_nunique_title'] = train_data.groupby('query').title.transform('nunique')
train_data['title_nunique_prefix'] = train_data.groupby('title').prefix.transform('nunique')


######下面这段代码最终被弃用，
#######原因，apply实际上就相当于一个for循环！！！如果你不需要做一些特殊的操作，而是只需要做一些加减乘除max std
######建议用比如 df['第一列']+df['第二列'] 这样写，会比用apply(lambda x:x[0]+x[1])快几百倍（数据量大时）
for feature in [max, min, np.mean, np.std]
    train_data["similarity_"+feature.__name__] = train_data[['title_query_dot_similarity', 'title_query_norm_similarity','title_query_cosine_similarity'].apply(lambda x:feature([x[0],x[1],x[2]]),axis=1)



# 统计特征（除CTR特征）

In [8]:
eooch_tongji=0

#query是否在title中
def get_is_query_in_title_feature(df):
    global eooch_tongji
    eooch_tongji+=1
    if eooch_tongji%5000==0:
        sys.stdout.write(str(eooch_tongji) + '\r')
        sys.stdout.flush()
    x = df['query']
    y = df['title']
    is_query_in_title = np.nan
    if x in y:
        is_query_in_title = 1
    else:
        is_query_in_title = 0
    return is_query_in_title

# 统计一些交叉种类特征
###################################这个特征是在别的大佬那里淘来的，
#################就是！！！把测试集和训练集合一起，然后统计 每个title下query的数量
##########该特征会造成严重的过拟合！！！！！！
def get_jiaocha_type_feature(train_df, valid_df, jiaocha_type_list):
    global eooch_tongji
    eooch_tongji+=1
    if eooch_tongji%5000==0:
        sys.stdout.write(str(eooch_tongji) + '\r')
        sys.stdout.flush()

    for jiaocha_type in jiaocha_type_list:
        fea1 = jiaocha_type[0]
        fea2 = jiaocha_type[1]
        temp_df = pd.concat([train_df, valid_df])
        temp_pivot_table = pd.pivot_table(temp_df[[fea1, fea2, 'label']], index=[fea1, fea2], values='label', aggfunc=len)
        temp_pivot_table.reset_index(inplace=True)
        final_pivot_table = pd.pivot_table(temp_pivot_table, index=fea1, values=fea2, aggfunc=len)
        final_pivot_table.reset_index(inplace=True)
        final_pivot_table.rename(columns={fea2 : fea1 + '_' + fea2 + '_types'}, inplace=True)
        train_df = pd.merge(train_df, final_pivot_table[[fea1, fea1 + '_' + fea2 + '_types']], on=fea1, how='left')
        valid_df = pd.merge(valid_df, final_pivot_table[[fea1, fea1 + '_' + fea2 + '_types']], on=fea1, how='left')
    return train_df, valid_df

# 统计一些跟字符串长度相关的特征
###########################初赛的时候我傻傻的按照字级别来统计的句子长度
#########也就是统计了字的数量， 比如  251214  12472 126  这三个词组成的 query，我统计出来的长度是6+5+3=14
##############这个特征的重要性意外的高！！！！，甚至在我把 分词之后按词数量统计的长度加上之后
########这个特征的重要性甚至比按词统计长度 的特征重要性要高1倍
#######但是实际上文本已经被脱敏了，词（比如251214）中的251214 数字的大小不能代表词的长度
########至今我也没有想明白为什么这个特征的重要性会高
def get_string_len_feature(df):
    global eooch_tongji
    eooch_tongji+=1
    if eooch_tongji%5000==0:
        sys.stdout.write(str(eooch_tongji) + '\r')
        sys.stdout.flush()
    df['query_len'] = df['query'].map(lambda x : len(x))
    df['title_len'] = df['title'].map(lambda x : len(x))
    df['len_title-query'] = df['title_len'] - df['query_len']
    df['len_query/title'] = df['query_len'] / df['title_len']
    return df



# 统计title跟query的编辑距离
def get_title_query_levenshtein_distance(df):
    global eooch_tongji
    eooch_tongji+=1
    if eooch_tongji%5000==0:
        sys.stdout.write(str(eooch_tongji) + '\r')
        sys.stdout.flush()
    title = df['title']
    query = df['query']
    return Levenshtein.distance(title, query)

#实际上就是用title的长度做了一个平滑！！！
def get_title_query_levenshtein_distance_rate(df):
    global eooch_tongji
    eooch_tongji+=1
    if eooch_tongji%5000==0:
        sys.stdout.write(str(eooch_tongji) + '\r')
        sys.stdout.flush()
    title_query_leven = df['title_query_leven']
    title = df['title']
    return (title_query_leven / (len(title) + 3))



# query/title 共现词之类的特征统计

### 泡面姐说fuzzy是个自杀库，我以为不好使，比赛结束哭了，还是得自己试验！
### 用了fuzzy的大佬们，他们的共现词特征重要度都出奇的高，但我的共现词重要度都忽上忽下

In [9]:
epoch=0
def word_match_share(df):
    q1words = {}
    q2words = {}
    for word in df[0]:
        q1words[word] = 1
    for word in df[1]:
        q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    global epoch
    epoch+=1
    if epoch%5000==0:
        sys.stdout.write(str(epoch) + '\r')
        sys.stdout.flush()
    return R



def jaccard(df):
    wic = set(df[0]).intersection(set(df[1]))
    uw = set(df[0]).union(df[1])
    if len(uw) == 0:
        uw = [1]
    global epoch
    epoch+=1
    if epoch%5000==0:
        sys.stdout.write(str(epoch) + '\r')
        sys.stdout.flush()
    return (len(wic) / len(uw))



def common_words(df):
    global epoch
    epoch+=1
    if epoch%5000==0:
        sys.stdout.write(str(epoch) + '\r')
        sys.stdout.flush()
    return len(set(df[0]).intersection(set(df[1])))



def total_unique_words(df):
    global epoch
    epoch+=1
    if epoch%5000==0:
        sys.stdout.write(str(epoch) + '\r')
        sys.stdout.flush()
    return len(set(df[0]).union(df[1]))



def wc_diff(df):
    global epoch
    epoch+=1
    if epoch%5000==0:
        sys.stdout.write(str(epoch) + '\r')
        sys.stdout.flush()
    return abs(len(df[0]) - len(df[1]))



def wc_ratio(df):
    global epoch
    epoch+=1
    if epoch%5000==0:
        sys.stdout.write(str(epoch) + '\r')
        sys.stdout.flush()
    l1 = len(df[0])*1.0
    l2 = len(df[1])
    if l2 == 0:
        return np.nan

    if l1 / l2:
        return l2 / l1

    else:
        return l1 / l2



def wc_diff_unique(df):
    global epoch
    epoch+=1
    if epoch%5000==0:
        sys.stdout.write(str(epoch) + '\r')
        sys.stdout.flush()
    return abs(len(set(df[0])) - len(set(df[1])))



def wc_ratio_unique(df):
    global epoch
    epoch+=1
    if epoch%5000==0:
        sys.stdout.write(str(epoch) + '\r')
        sys.stdout.flush()
    l1 = len(set(df[0])) * 1.0
    l2 = len(set(df[1]))
    if l2 == 0:
        return np.nan
    if l1 / l2:
        return l2 / l1
    else:
        return l1 / l2





def deal_word_for_all(train_df, fea1, fea2, func, colName):
    train_df[colName] = train_df[[fea1, fea2]].apply(func, axis=1)
#     valid_df[colName] = valid_df[[fea1, fea2]].apply(func, axis=1)
    print(colName + ' finish!!!')
    return train_df


def get_word_statistic_feature(train_df, col_list):
    for col in col_list:
        fea1 = col[0]
        fea2 = col[1]
        train_df = deal_word_for_all(train_df, fea1, fea2, word_match_share, fea1[0] + '_' + fea2[0] + '_word_match')
        train_df = deal_word_for_all(train_df, fea1, fea2, common_words, fea1[0] + '_' + fea2[0] + '_common_words')
        train_df = deal_word_for_all(train_df, fea1, fea2, total_unique_words, fea1[0] + '_' + fea2[0] + '_total_unique_words')
        train_df = deal_word_for_all(train_df, fea1, fea2, wc_diff, fea1[0] + '_' + fea2[0] + '_wc_diff')
        train_df = deal_word_for_all(train_df, fea1, fea2, wc_ratio, fea1[0] + '_' + fea2[0] + '_wc_ratio')
        train_df = deal_word_for_all(train_df, fea1, fea2, wc_diff_unique, fea1[0] + '_' + fea2[0] + '_wc_diff_unique')
        train_df = deal_word_for_all(train_df, fea1, fea2, wc_ratio_unique, fea1[0] + '_' + fea2[0] + '_wc_ratio_unique')
#         f = functools.partial(tfidf_word_match_share, weights=weights)
#         train_df, valid_df = deal_word_for_all(train_df, valid_df, fea1, fea2, f, fea1[0] + '_' + fea2[0] + '_tfidf_word_match_share')
    return train_df

# 基于词向量的query title相似度

In [5]:
import numpy as np
import pandas as pd
import time
import sys
import datetime
import gc
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from time import time
import Levenshtein
from gensim.models import Word2Vec
from time import sleep
data_path='/home/kesci/input/bytedance/first-round/'
data_store_path='/home/kesci/work/test/'


epoch_w2v=0

def get_w2v_feature(word_list, word_wv, w2v_size):
    word_list[0]=word_list[0].split(' ')
    word_list[1]=word_list[1].split(' ')
    
    
    wmd_dist = w2v_model.wmdistance(word_list[0], word_list[1])
    
    if wmd_dist==np.inf:
        wmd_dist=np.nan
    
    global epoch_w2v
    epoch_w2v+=1
    if epoch_w2v%5000==0:
        sys.stdout.write(str(epoch_w2v) + '\r')
        sys.stdout.flush()
    word_vectors_query = np.zeros((len(word_list[0]), w2v_size))
    word_vectors_title = np.zeros((len(word_list[1]), w2v_size))
    
    for i in range(len(word_list[0])):
        if str(word_list[0][i]) in word_wv.vocab.keys():
            word_vectors_query[i][:] = word_wv[str(word_list[0][i])]
    for i in range(len(word_list[1])):
        if str(word_list[1][i]) in word_wv.vocab.keys():
            word_vectors_title[i][:] = word_wv[str(word_list[1][i])]

    mean_array_query = np.mean(word_vectors_query, axis=0)
    mean_array_title = np.mean(word_vectors_title, axis=0)
    ####################以上是句子向量的做法
    ############就是先初始化一个全0矩阵，shape是：句子长度（即词个数）*每个词embedding_size
    ###########然后每个词对应和行，填充词对应的embedding。
    ############最后竖向求平均。
    
    ######下面是向量距离的做法
    dot_similarity = np.dot(mean_array_title, mean_array_query)
    norm_similarity = np.linalg.norm(mean_array_title - mean_array_query)
    cosine_similarity=np.dot(mean_array_title,mean_array_query) / (np.linalg.norm(mean_array_title) * np.linalg.norm(mean_array_query))
    return [dot_similarity,norm_similarity,cosine_similarity,wmd_dist]
    


print("~~~~~~~~~~~~~~~~~~~~~~测试集词向量特征~~~~~~~~~~~~~~~~~~~~~~~~~~")
import gc

w2v_model = Word2Vec.load('w2v_model/w2v_all_data_model.txt')
word_wv = w2v_model.wv

train_data=pd.read_csv("/home/kesci/work/1000W_stacking/data_sets/test_data_ctr.csv",usecols=['query','title'])#读已经做好ctr特征的训练集
train_data = train_data.apply(lambda x : get_w2v_feature(x, word_wv, w2v_model.vector_size),axis=1,result_type='expand')
train_data.columns=['title_query_dot_similarity', 'title_query_norm_similarity','title_query_cosine_similarity','title_query_wmd_dis']
train_data.to_csv("/home/kesci/work/1000W_stacking/data_sets/w2v_feature_test.csv",index=None)
del train_data



# 以下是main

In [6]:
print("~~~~~~~~~~~~~~~~~~~~~~交叉特征~~~~~~~~~~~~~~~~~~~~~~~~~~")
time1=time()
train_data['is_query_in_title'] = train_data[['query', 'title']].apply(get_is_query_in_title_feature, axis = 1)
test_data['is_query_in_title'] = test_data[['query', 'title']].apply(get_is_query_in_title_feature, axis = 1)
jiaocha_type_list = [['query', 'title']]
train_data, test_data = get_jiaocha_type_feature(train_data, test_data, jiaocha_type_list)
train_data = get_string_len_feature(train_data)
test_data = get_string_len_feature(test_data)
time2=time()
print("时间：",time2-time1)
print("~~~~~~~~~~~~~~~~~~~~~~编辑距离特征~~~~~~~~~~~~~~~~~~~~~~~~~~")
train_data['title_query_leven'] = train_data[['title', 'query']].apply(get_title_query_levenshtein_distance, axis=1)
train_data['title_query_leven_rate'] = train_data[['title', 'title_query_leven']].apply(get_title_query_levenshtein_distance_rate, axis=1)
test_data['title_query_leven'] = test_data[['title', 'query']].apply(get_title_query_levenshtein_distance, axis=1)
test_data['title_query_leven_rate'] = test_data[['title', 'title_query_leven']].apply(get_title_query_levenshtein_distance_rate, axis=1)
time3=time()
print("时间：",time3-time2)
print("~~~~~~~~~~~~~~~~~~~~~~距离特征~~~~~~~~~~~~~~~~~~~~~~~~~~")
train_data[['query']]=train_data[['query']].applymap(lambda x:x.split(' '))
train_data[['title']]=train_data[['title']].applymap(lambda x:x.split(' '))


time4=time()
print("转成list词表时间：",time4-time3)
col_list = [ ['query', 'title']]
train_data = get_word_statistic_feature(train_data, col_list)
train_data.to_csv(data_store_path+"data_sets/train_feature_second_data.csv",index=None)
train_data=train_data.drop(['query','title'],axis=1)
train_data.to_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv",index=None)
del train_data
import gc
gc.collect()
test_data[['query']]=test_data[['query']].applymap(lambda x:x.split(' '))
test_data[['title']]=test_data[['title']].applymap(lambda x:x.split(' '))
test_data = get_word_statistic_feature(test_data, col_list)
test_data.to_csv(data_store_path+"data_sets/test_feature_second_data.csv",index=None)
test_data=test_data.drop(['query','title'],axis=1)
test_data.to_csv(data_store_path+"data_sets/test_feature_second_no_query_and_title_data.csv",index=None)
del test_data
gc.collect()
time5=time()
print("距离特征时间：",time5-time4)
# print("~~~~~~~~~~~~~~~~~~~~~~词向量特征~~~~~~~~~~~~~~~~~~~~~~~~~~")
# w2v_model = Word2Vec.load('w2v_all_data_model.txt')
# word_wv = w2v_model.wv
# train_data['title_array'] = train_data['title'].map(lambda x : get_w2v_array(x, word_wv, w2v_model.vector_size))
# train_data['query_array'] = train_data['query'].map(lambda x : get_w2v_array(x, word_wv, w2v_model.vector_size))
# test_data['title_array'] = test_data['title'].map(lambda x : get_w2v_array(x, word_wv, w2v_model.vector_size))
# test_data['query_array'] = test_data['query'].map(lambda x : get_w2v_array(x, word_wv, w2v_model.vector_size))
# train_data = get_similarity_feature(train_data)
# test_data = get_similarity_feature(test_data)



~~~~~~~~~~~~~~~~~~~~~~交叉特征~~~~~~~~~~~~~~~~~~~~~~~~~~


NameError: name 'train_data' is not defined

# 下面是把已经做好的特征read

In [ ]:
train_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
test_data=pd.read_csv(data_store_path+"data_sets/test_feature_second_no_query_and_title_data.csv")
train_data_similarity=pd.read_csv(data_store_path+"word2vec_feature_parts/train_data_second_similarity.csv")
test_data_similarity=pd.read_csv(data_store_path+"word2vec_feature_parts/test_data_second_similarity.csv")
train_data=pd.concat([train_data,train_data_similarity],axis=1)
test_data=pd.concat([test_data,train_data_similarity],axis=1)
del train_data_similarity
del test_data_similarity
import gc
gc.collect()

# 训练

In [8]:

fea=list(train_data.columns)
fea.remove('label')
data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.02,random_state=2018)
train_index,vaild_index=data_split.split(train_data['label'],train_data['label']).__next__()
X_vaild,y_vaild=train_data[fea].iloc[vaild_index],train_data['label'].iloc[vaild_index]
params = {
        'boosting_type':'gbdt', 'num_leaves':127, 'max_depth':-1, 'n_estimators':5000, 'objective':'binary',
        'subsample':0.8, 'colsample_bytree':1, 'subsample_freq':1,
        'learning_rate':0.008, 'random_state':666
        ,  'num_boost_round':500,'silent':False,'verbose':2
}

lgb_train = lgb.Dataset(train_data[fea], train_data['label'],categorical_feature=["query_tfidf_kmeans"])
lgb_eval = lgb.Dataset(X_vaild, y_vaild, reference=lgb_train,categorical_feature=["query_tfidf_kmeans"])
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=530,
                valid_sets=lgb_eval,
                verbose_eval=2,categorical_feature=["query_tfidf_kmeans"]
                
                #init_model=data_store_path+'/lgb_model/model_1000.txt'
                )#early_stopping_rounds=20,
gbm.save_model(data_store_path+'/lgb_model/model_1130.txt')
print(gbm.best_score['valid_0'])

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields query, title

# 输出模型训练后的一些信息（不包括预测）

In [21]:
for index,i in enumerate(fea):
    print(i,":",gbm.feature_importance()[index])

title_count : 6101
title_ctr : 11374
title_sum : 5211
is_query_in_title : 1815
query_title_types : 4530
query_len : 15679
title_len : 4014
len_title-query : 1591
len_query/title : 9399
title_query_leven : 2717
title_query_leven_rate : 7064
q_t_word_match : 7733
q_t_common_words : 5263
q_t_total_unique_words : 7286
q_t_wc_diff : 1403
q_t_wc_ratio : 8594
q_t_wc_diff_unique : 995
q_t_wc_ratio_unique : 9391
title_query_dot_similarity : 10086
title_query_norm_similarity : 11233
title_query_cosine_similarity : 10901


# 保存模型

In [8]:
gbm.save_model(data_store_path+'/lgb_model/model.txt')
print(gbm.best_score['valid_0'])

{'binary_logloss': 0.5336765283668315}


# 预测+结果平滑

## 结果平滑(这个没什么用，因为query中train和test中重复的很少)
## 使用的是阈值平滑

In [36]:
# ############平滑函数
# #定义调整函数

# def resultAdjustment(result_df, t):
#     result_df_temp = result_df.copy()
#     result_df_temp['x'] = result_df_temp.predicted_score.map(lambda x: -(math.log(((1 - x) / x), math.e)))
#     result_df_temp['adjust_result'] = result_df_temp.x.map(lambda x: 1 / (1 + math.exp(-(x + t))))
#     print(result_df_temp['adjust_result'].mean())
#     return result_df_temp['adjust_result']

In [35]:
# query_train_list=[]
# for epoch,part_data in enumerate(pd.read_csv(data_path+"train.csv",names=['query_id','query','query_title_id','title','label'],usecols=[1,3,4],chunksize=1000000,header=None)):
#     all_data_train_list.append(part_data)
#     print(epoch)
# all_data_train=pd.concat(all_data_train_list,axis=0)
# del all_data_train_list

# ###################此处为一个trick，就是给结果平滑一下
# y_pred_df
# is_query_in_title_feature=pd.read_csv('test_feature_first_no_query_and_title_data.csv',sep=',',usecols=['is_query_in_title'])
# y_pred_df_query_in=pd.concat([y_pred_df,is_query_in_title_feature],axis=1)
# y_pred_df_query_in
# ########################加载进了 is_query_in_title_feature
# ######下面是平滑操作


,0,is_query_in_title
0,0.121394,0
1,0.123962,0
2,0.194063,0
3,0.118342,0
4,0.302090,0
5,0.295319,0
6,0.254328,0
7,0.074963,0
8,0.093010,0
9,0.089078,0


# 这个是加载已训练lgb模型

In [33]:
# model2=lgb.Booster(model_file='model_first.txt')  #init model
# model2.feature_name()

## 预测

In [22]:
y_pred = gbm.predict(test_data[fea], num_iteration=gbm.best_iteration)
y_pred_df=pd.DataFrame(y_pred)
y_pred_df.to_csv(data_store_path+"sub/result_1130.csv",index=None)
y_pred_df######这个结果只是单列结果，需要和query_id和title_id拼接后提交

,0
0,0.124806
1,0.134846
2,0.193158
3,0.122989
4,0.343902
5,0.311168
6,0.279804
7,0.069874
8,0.070957
9,0.062102


In [23]:
all_data_test_list_pred_need=[]
for epoch,part_data in enumerate(pd.read_csv(data_path+'test.csv',sep=',',names=['query_id','query','query_title_id','title'],usecols=['query_id','query_title_id'],chunksize=1000000,header=None)):
    all_data_test_list_pred_need.append(part_data)
test_data_pred_need=pd.concat(all_data_test_list_pred_need,axis=0)
#,usecols=[1,3]
test_data_pred_need#整出query_id和title_id

,query_id,query_title_id
0,1,3
1,1,1
2,1,4
3,1,2
4,2,1
5,2,2
6,2,3
7,3,6
8,3,5
9,3,8


# 提交sub

In [36]:

result=pd.concat([test_data_pred_need,y_pred_df],axis=1)
result.to_csv(data_store_path+"sub/sub_1130_rong_700.csv",header=None,index=None)#####合成三列做 最终的提交结果
# y_pred_df
# result[0]
result

,query_id,query_title_id,0
0,1,3,0.124806
1,1,1,0.134846
2,1,4,0.193158
3,1,2,0.122989
4,2,1,0.343902
5,2,2,0.311168
6,2,3,0.279804
7,3,6,0.069874
8,3,5,0.070957
9,3,8,0.062102


In [37]:
!wget -nv -O kesci_submit https://www.heywhale.com/kesci_submit&&chmod +x kesci_submit
!./kesci_submit -token 03111ac783b57b48 -file /home/kesci/work/1000Wlgb/sub/sub_1130_rong_700.csv

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
2019-06-14 03:22:49 URL:https://www.heywhale.com/kesci_submit [7842088/7842088] -> "kesci_submit" [1]
Kesci Submit Tool
Result File: /home/kesci/work/1000Wlgb/sub/sub_1130_rong_700.csv (136.77 MiB)
Uploaded.       
Submit Success.
{"Stage":0,"Status":0,"ShownInHistory":true,"IsAucResult":true,"Selected":false,"_id":"5d031318902bad002c294e02","Competition":"5cc51043f71088002c5b8840","Team":"5cea3ae3cde7ed002b540c8d","UploadDate":"2019-06-14T03:23:04.381Z","Final":true,"Response":"","SubmissionResults":[],"IP":"52.82.19.99","FingerPrint":"","UserAgent":"Go-http-client/1.1","ResultFileName":"1560482571202c6f049.csv","ResultFileRealName":"sub_1130_rong_700.csv","ResultFileSize":0,"ReviewInfos":[],"__v":0}

